# 1. Scraping Zomato Website

In [ ]:
#Importing packages 
import pandas as pd
import selenium
from selenium import webdriver
import time

In [ ]:
#download chromedriver.exe and save it in your current directory
browser = webdriver.Chrome("chromedriver.exe")
browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [ ]:
df_1 = pd.DataFrame(columns=['Restaurant Name','URL to the restaurant'])

In [ ]:
parent_tag = browser.find_elements_by_css_selector('div.h100.pb20')
len(parent_tag)

In [ ]:
#Function defination that will return 1st dataframe
def get_url(parent_tag):
    df_1 = pd.DataFrame(columns=['Restaurant Name','URL to the restaurant'])
    for each_restaurment in parent_tag:
        try:
            restaurant_name = each_restaurment.find_element_by_css_selector('div.res_title.zblack.bold.nowrap'). \
                              get_attribute('innerHTML').split('\n')[1]
        except:
            restaurant_name = each_restaurment.find_element_by_css_selector('div.res_title.zblack.bold'). \
                              get_attribute('innerHTML').split('\n')[1]

        URL = each_restaurment.find_element_by_css_selector('a').get_attribute('href')
        
        curr_row = {'Restaurant Name':restaurant_name,
                    'URL to the restaurant':URL}
            
        df_1=df_1.append(curr_row,ignore_index=True)
    return df_1

In [ ]:
#calling the above function that will return a dataframe with all the Restaurant Names and their respective URLs 
df = get_url(parent_tag)

In [ ]:
df.head()

In [ ]:
#Function defination that will return final dataframe
def get_ratings(data):
    df_2 = pd.DataFrame(columns=['Restaurant Name','Restaurant ID','User Name','User ID','Rating'])
    df_3 = pd.DataFrame(columns=['Restaurant Name','URL to the restaurant'])
    for each_row in data.values:
        browser.get(each_row[1])
        
        if len(browser.find_elements_by_css_selector('div.ui.segments.res-review-body.res-review.clearfix.js-activity-root.mbti.item-to-hide-parent.stupendousact'))!=0:
            try:
                while(browser.find_element_by_css_selector('div.load-more.bold.ttupper.tac.cursor-pointer.fontsize2')!=None):
                    browser.find_element_by_css_selector('div.load-more.bold.ttupper.tac.cursor-pointer.fontsize2').click()
                    time.sleep(3)
            except:
                res_ID = browser.find_element_by_css_selector('div.res-rating.pos-relative.clearfix.mb5').find_element_by_css_selector('div[data-res-id]').get_attribute('data-res-id')
                res_name = each_row[0]+','+browser.find_element_by_css_selector('div.mb5.pt5.clear').find_element_by_css_selector('a').get_attribute('innerText')
                users = browser.find_elements_by_css_selector('div.ui.segments.res-review-body.res-review.clearfix.js-activity-root.mbti.item-to-hide-parent.stupendousact')
                
                for each_user in users:
                    user_name = each_user.find_element_by_css_selector('div.header.nowrap.ui.left').get_attribute('innerText')
                    rate = each_user.find_element_by_css_selector('div[aria-label]').get_attribute('aria-label').split(' ')[1]
                    user_ID = each_user.find_element_by_css_selector('a[data-entity_id]').get_attribute('data-entity_id')
                    curr_row = [{'Restaurant Name':res_name,'Restaurant ID':res_ID,'User Name':user_name,'User ID':user_ID,'Rating':rate}]
                    df_2 = df_2.append(curr_row,ignore_index = True)
        else:
            res_name = each_row[0]
            URL_branch = browser.find_elements_by_css_selector('div.card.search-snippet-card.search-card')
            for url in URL_branch:
                branch_url = url.find_element_by_css_selector('a.result-title.hover_feedback.zred.bold.ln24.fontsize0').get_attribute('href')
                curr_row = [{'Restaurant Name':res_name,'URL to the restaurant':branch_url}]
                df_3 = df_3.append(curr_row, ignore_index=True)
    if not df_3.empty:
        return df_2,df_3
    else:
        return df_2

In [ ]:
#calling the above function that will return all the details related to restaurants(Res Name+location,
#Res Id, User Name, User ID, Rating)
df_4 = get_ratings(df)

In [ ]:
#calling the above function that will return all the details related to restaurants(Res Name+location,
#Res Id, User Name, User ID, Rating)
df_4 = get_ratings(df)

In [ ]:
#convert the dataframe into a csv file named 'Zomato_final.csv'
try:
    df_5 = get_ratings(df_4[1])
    df_final = pd.concat([df_4[0],df_5], axis =0, ignore_index = True)
    df_final.to_csv('Zomato_final.csv',index = False)
except:
    df_4.to_csv('Zomato_final.csv',index = False)
    